<center>   
<h3> </h3>

<h1>Hierarchical Attention Network Using GRU</h1>


</center>
In this lab, we will get familiar with recurrent neural networks (RNNs), self-attention, and the HAN architecture <b>(Yang et al. 2016)</b> using PyTorch. In this architecture, sentence embeddings are first individually produced, and a document embedding is then computed from the sentence embeddings.<br>



### = = = = =  Attention Layer = = = = =
In thi section, we will implement the self-attention layer. This layer will be used later to define the HAN architecture. The basic idea behind attention is that rather than considering the last annotation $h_T$ as a summary of the entire sequence, which is prone to information loss, the annotations at <i>all</i> time steps are used.
The self-attention mechanism computes a weighted sum of the annotations, where the weights are determined by trainable parameters.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls

data.zip  drive  sample_data


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader

class AttentionWithContext(nn.Module):
    """
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    """

    def __init__(self, input_shape, return_coefficients=False, bias=True):
        super(AttentionWithContext, self).__init__()
        self.return_coefficients = return_coefficients

        self.W = nn.Linear(input_shape, input_shape, bias=bias)
        self.tanh = nn.Tanh()
        self.u = nn.Linear(input_shape, 1, bias=False)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.W.weight.data.uniform_(-initrange, initrange)
        self.W.bias.data.uniform_(-initrange, initrange)
        self.u.weight.data.uniform_(-initrange, initrange)

    def generate_square_subsequent_mask(self, sz):
        # do not pass the mask to the next layers
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = (
            mask.float()
            .masked_fill(mask == 0, float("-inf"))
            .masked_fill(mask == 1, float(0.0))
        )
        return mask

    def forward(self, x, mask=None):
        uit = self.W(x)        # compute uit = W . x  where x represents ht
        uit = self.tanh(uit)
        ait = self.u(uit)
        a = torch.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            a = a * mask.double()

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        eps = 1e-9
        a = a / (torch.sum(a, axis=1, keepdim=True) + eps)
        weighted_input =  a  * x
        s = torch.sum(weighted_input,axis = 1)                 # the attentional vector
        if self.return_coefficients:
            return  s,a      ### [attentional vector, coefficients]
        else:
            return  s

### = = = = = Parameters = = = = =
In this section, we define the parameters to use in our training. Such as data path, the embedding dimention <b>d</b>, the GRU layer dimensionality <b>n_units</b>, etc..<br>
The parameter <b>device</b> is used to train the model on GPU if it is available. for this purpose, if you are using Google Colab, switch your runtime to a GPU runtime to train the model with a maximum speed.<br>


In [ ]:
import sys
import json
import operator
import numpy as np

path_root = ''
path_to_data = path_root + 'data/'

d = 30 # dimensionality of word embeddings
n_units = 50 # RNN layer dimensionality
drop_rate = 0.5 # dropout
mfw_idx = 2 # index of the most frequent words in the dictionary
            # 0 is for the special padding token
            # 1 is for the special out-of-vocabulary token

padding_idx = 0
oov_idx = 1
batch_size = 64
nb_epochs = 15
my_patience = 2 # for early stopping strategy (used to sentinel accuracy evolution in validation set)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### = = = = = Data Loading = = = = =
In this section we will use first <b>wget</b> to download the data the we will load it using numpy in the first cell. While in the second cell, we will use these data to define our Pytorch data loader. Note that the data is already preprocessed, tokenized and padded.<br><br>
<b>Note: if you are running your notebook on Windows or on MacOS, <i>wget</i> will probably not work if you did not install it manually. In this case, use the provided link to download the data and change the <i>path_to_data</i> in the <i>Parameters</i> section accordingly. Otherwise, you will face no problem on Ubuntu and Google Colab.</b>



In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: __MACOSX/._data         
  inflating: data/labels_train.npy   
  inflating: __MACOSX/data/._labels_train.npy  
  inflating: data/docs_test.npy      
  inflating: __MACOSX/data/._docs_test.npy  
  inflating: data/labels_test.npy    
  inflating: __MACOSX/data/._labels_test.npy  
  inflating: data/word_to_index.json  
  inflating: __MACOSX/data/._word_to_index.json  
  inflating: data/docs_train.npy     
  inflating: __MACOSX/data/._docs_train.npy  


In [ ]:
my_docs_array_train = np.load(path_to_data + 'docs_train.npy')
my_docs_array_test = np.load(path_to_data + 'docs_test.npy')

my_labels_array_train = np.load(path_to_data + 'labels_train.npy')
my_labels_array_test = np.load(path_to_data + 'labels_test.npy')

# load dictionary of word indexes (sorted by decreasing frequency across the corpus)
with open(path_to_data + 'word_to_index.json', 'r') as my_file:
    word_to_index = json.load(my_file)

# invert mapping
index_to_word =  {idx:word for word, idx in word_to_index.items()}
index_to_word[0] = "<PAD>"
index_to_word[1] = "<OOV>"
input_size = my_docs_array_train.shape

print("Train docs max index:", my_docs_array_train.max())
print("Test docs max index:", my_docs_array_test.max())


vocab_size = len(index_to_word.keys())
print("Vocab size:", vocab_size)

test_emb = torch.nn.Embedding(vocab_size, d, padding_idx=0).to("cuda")
print("✅ Embedding ok!")

Train docs max index: 29937
Test docs max index: 29937
Vocab size: 29938
✅ Embedding ok!


In [ ]:
import numpy
import torch
from torch.utils.data import DataLoader, Dataset


class Dataset_(Dataset):
    def __init__(self, x, y):
        self.documents = x
        self.labels = y

    def __len__(self):
        return len(self.documents)

    def __getitem__(self, index):
        document = self.documents[index]
        label = self.labels[index]
        sample = {
            "document": torch.tensor(document),
            "label": torch.tensor(label),
            }
        return sample


def get_loader(x, y, batch_size=32):
    dataset = Dataset_(x, y)
    data_loader = DataLoader(dataset=dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            pin_memory=True,
                            drop_last=True,
                            )
    return data_loader

### = = = = = Defining Architecture = = = = =
In this section, we define the HAN architecture. We start with <i>AttentionBiGRU</i> module in order to define the sentence encoder (check Figure 3 in the handout). Then, we define the <i>TimeDistributed</i> module to allow us to forward our input (batch of document) as to the sentence encoder as <b>batch of sentences</b>, where each sentence in the document will be considered as a time step. This module also reshape the output to a batch of timesteps representations per document. Finally we define the <b>HAN</b> architecture using <i>TimeDistributed</i>, <i>AttentionWithContext</i> and <i>GRU</i>.



In [ ]:

class AttentionBiGRU(nn.Module):
    def __init__(self, input_shape, n_units, index_to_word, dropout=0):
        super(AttentionBiGRU, self).__init__()
        self.embedding = nn.Embedding(len(index_to_word), # vocab size
                                      d, # dimensionality of embedding space
                                      padding_idx=0)
        self.dropout = nn.Dropout(drop_rate)
        self.gru = nn.GRU(input_size=d,
                          hidden_size=n_units,
                          num_layers=1,
                          bias=True,
                          batch_first=True,
                          bidirectional=True)
        self.attention = AttentionWithContext(input_shape,    # the input shape for the attention layer
                                              return_coefficients=True)


    def forward(self, sent_ints):
        sent_wv = self.embedding(sent_ints)
        sent_wv_dr = self.dropout(sent_wv)
        sent_wa, _ = self.gru(sent_wv_dr) # RNN layer
        sent_att_vec, word_att_coeffs = self.attention(sent_wa) # attentional vector for the sent
        sent_att_vec_dr = self.dropout(sent_att_vec)
        return sent_att_vec_dr, word_att_coeffs

class TimeDistributed(nn.Module):
    def __init__(self, module, batch_first=False):
        super(TimeDistributed, self).__init__()
        self.module = module
        self.batch_first = batch_first

    def forward(self, x):
        if len(x.size()) <= 2:
            return self.module(x)
        # Squash samples and timesteps into a single axis
        x_reshape = x.contiguous().view(-1, x.size(-1))  # (samples * timesteps, input_size) (448, 30)
        sent_att_vec_dr, word_att_coeffs = self.module(x_reshape)
        # We have to reshape the output
        if self.batch_first:
            sent_att_vec_dr = sent_att_vec_dr.contiguous().view(x.size(0), -1, sent_att_vec_dr.size(-1))  # (samples, timesteps, output_size)
            word_att_coeffs = word_att_coeffs.contiguous().view(x.size(0), -1, word_att_coeffs.size(-1))  # (samples, timesteps, output_size)
        else:
            sent_att_vec_dr = sent_att_vec_dr.view(-1, x.size(1), sent_att_vec_dr.size(-1))  # (timesteps, samples, output_size)
            word_att_coeffs = word_att_coeffs.view(-1, x.size(1), word_att_coeffs.size(-1))  # (timesteps, samples, output_size)
        return sent_att_vec_dr, word_att_coeffs

class HAN(nn.Module):
    def __init__(self, input_shape, n_units, index_to_word, dropout=0):
        super(HAN, self).__init__()
        self.encoder = AttentionBiGRU(input_shape, n_units, index_to_word, dropout)
        self.timeDistributed = TimeDistributed(self.encoder, True)
        self.dropout = nn.Dropout(drop_rate)
        self.gru = nn.GRU(input_size= 2*n_units   ,     # the input shape of GRU layer
                          hidden_size=n_units,
                          num_layers=1,
                          bias=True,
                          batch_first=True,
                          bidirectional=True)
        self.attention = AttentionWithContext(2 * n_units,  # the input shape of between-sentence attention layer
                                              return_coefficients=True)
        self.lin_out = nn.Linear(2 * n_units,    # the input size of the last linear layer
                                 1)
        self.preds = nn.Sigmoid()

    def forward(self, doc_ints):
        sent_att_vecs_dr, word_att_coeffs = self.timeDistributed(doc_ints)
        doc_sa, _ = self.gru(sent_att_vecs_dr)
        doc_att_vec, sent_att_coeffs = self.attention(doc_sa)
        doc_att_vec_dr = self.dropout(doc_att_vec)
        doc_att_vec_dr = self.lin_out(doc_att_vec_dr)
        return self.preds(doc_att_vec_dr), word_att_coeffs, sent_att_coeffs


### = = = = = Training = = = = =
In this section, we have two code cells. In the first one, we define our evaluation function to compute the training and validation accuracies. While in the second one, we define our model, loss and optimizer and train the model over <i>nb_epochs</i>.<br>




In [ ]:
def evaluate_accuracy(data_loader, verbose=True):
    model.eval()
    total_loss = 0.0
    ncorrect = ntotal = 0
    with torch.no_grad():
        for idx, data in enumerate(data_loader):
            # inference
            output = model(data["document"].to(device))[0]
            output = output[:, -1] # only last vector
            # total number of examples
            ntotal +=  output.shape[0]
            # number of correct predictions
            predictions = torch.round(output)
            ncorrect += torch.sum(predictions == data["label"].to(device))
        acc = ncorrect.item() / ntotal
        if verbose:
          print("validation accuracy: {:3.2f}".format(acc*100))
        return acc

In [ ]:
print("len(index_to_word):", len(index_to_word))
print("type des clés:", type(list(index_to_word.keys())[0]))
print("min key:", min(index_to_word.keys()))
print("max key:", max(index_to_word.keys()))
print("0 in dict:", 0 in index_to_word)
print("1 in dict:", 1 in index_to_word)

len(index_to_word): 29938
type des clés: <class 'int'>
min key: 0
max key: 29937
0 in dict: True
1 in dict: True


In [ ]:
from tqdm import tqdm
model = HAN(2*n_units, n_units, index_to_word).to(device)
model = model.double()
lr = 0.001  # learning rate
criterion = nn.BCELoss() # use Binary cross entropy from torch.nn: https://pytorch.org/docs/stable/nn.html#loss-functions
optimizer = torch.optim.Adam(model.parameters(), lr=lr) #fill me

def train(x_train=my_docs_array_train,
          y_train=my_labels_array_train,
          x_test=my_docs_array_test,
          y_test=my_labels_array_test,
          word_dict=index_to_word,
          batch_size=batch_size):

    train_data = get_loader(x_train, y_train, batch_size)
    test_data = get_loader(my_docs_array_test, my_labels_array_test, batch_size)

    best_validation_acc = 0.0
    p = 0 # patience

    for epoch in range(1, nb_epochs + 1):
        losses = []
        accuracies = []
        with tqdm(train_data, unit="batch") as tepoch:
            for idx, data in enumerate(tepoch):
                tepoch.set_description(f"Epoch {epoch}")
                model.train()
                optimizer.zero_grad()
                # Cast input data to double
                input = data['document'].to(device)
                label = data['label'].to(device)
                label = label.double()
                output = model.forward(input)[0]
                output = output[:, -1]
                loss = criterion(output,label)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5) # prevent exploding gradient
                optimizer.step()

                losses.append(loss.item())
                accuracy = torch.sum(torch.round(output) == label).item() / batch_size
                accuracies.append(accuracy)
                tepoch.set_postfix(loss=sum(losses)/len(losses), accuracy=100. * sum(accuracies)/len(accuracies))

        # train_acc = evaluate_accuracy(train_data, False)
        test_acc = evaluate_accuracy(test_data, False)
        print("===> Epoch {} Complete: Avg. Loss: {:.4f}, Validation Accuracy: {:3.2f}%"
              .format(epoch, sum(losses)/len(losses), 100.*test_acc))
        if test_acc >= best_validation_acc:
            best_validation_acc = test_acc
            print("Validation accuracy improved, saving model...")
            torch.save(model.state_dict(), './best_model.pt')
            p = 0
            print()
        else:
            p += 1
            if p==my_patience:
                print("Validation accuracy did not improve for {} epochs, stopping training...".format(my_patience))
    print("Loading best checkpoint...")
    model.load_state_dict(torch.load('./best_model.pt'))
    model.eval()
    print('done.')

train()

Epoch 1: 100%|██████████| 390/390 [00:16<00:00, 23.47batch/s, accuracy=58.4, loss=0.664]


===> Epoch 1 Complete: Avg. Loss: 0.6636, Validation Accuracy: 69.25%
Validation accuracy improved, saving model...



Epoch 2: 100%|██████████| 390/390 [00:16<00:00, 23.62batch/s, accuracy=70.2, loss=0.57]


===> Epoch 2 Complete: Avg. Loss: 0.5696, Validation Accuracy: 75.68%
Validation accuracy improved, saving model...



Epoch 3: 100%|██████████| 390/390 [00:16<00:00, 24.12batch/s, accuracy=75.1, loss=0.506]


...

===> Epoch 13 Complete: Avg. Loss: 0.2923, Validation Accuracy: 84.59%
Validation accuracy did not improve for 2 epochs, stopping training...


Epoch 14: 100%|██████████| 390/390 [00:16<00:00, 24.28batch/s, accuracy=88.4, loss=0.281]


===> Epoch 14 Complete: Avg. Loss: 0.2810, Validation Accuracy: 84.02%


Epoch 15: 100%|██████████| 390/390 [00:16<00:00, 24.29batch/s, accuracy=88.9, loss=0.271]


===> Epoch 15 Complete: Avg. Loss: 0.2714, Validation Accuracy: 85.00%
Validation accuracy improved, saving model...

Loading best checkpoint...
done.


### = = = = = Extraction of Attention Coefficients = = = = =
In this section, we will extract and display the attention coefficients on two levels: sentence level and word level. To do so, we will extract the corresponding weights from our model.


In [ ]:
# select last review:
my_review = my_docs_array_test[-1:,:,:]

# convert integer review to text:
index_to_word[1] = '<OOV>'
my_review_text = [[index_to_word[idx] for idx in sent if idx in index_to_word] for sent in my_review.tolist()[0]]
print(my_review_text)

[['There', "'s", 'a', 'sign', 'on', 'The', 'Lost', 'Highway', 'that', 'says', ':', '<OOV>', 'SPOILERS', '<OOV>', '(', 'but', 'you', 'already', 'knew', 'that', ',', 'did', "n't", 'you', '?', ')', '<PAD>', '<PAD>', '<PAD>', '<PAD>'], ['Since', 'there', "'s", 'a', 'great', 'deal', 'of', 'people', 'that', 'apparently', 'did', 'not', 'get', 'the', 'point', 'of', 'this', 'movie', ',', 'I', "'d", 'like', 'to', 'contribute', 'my', 'interpretation', 'of', 'why', 'the', 'plot'], ['As', 'others', 'have', 'pointed', 'out', ',', 'one', 'single', 'viewing', 'of', 'this', 'movie', 'is', 'not', 'sufficient', '.', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>'], ['If', 'you', 'have', 'the', 'DVD', 'of', 'MD', ',', 'you', 'can', '<OOV>', "'", 'by', 'looking', 'at', 'David', 'Lynch', "'s", "'Top", '10', '<OOV>', 'to', '<OOV>', 'MD', "'", '(', 'but', 'only', 'upon', 'second'], [';', ')', 'First', 'of', 'all', ',', 'Mulholland', 

In [ ]:
x_pred = torch.tensor(my_review).to(device)
print(x_pred.shape)

torch.Size([1, 7, 30])


In [ ]:
x_pred

tensor([[[  130,    14,     6,  1991,    28,    22,  2746, 17943,    13,   564,
             85,     1,  3225,     1,    25,    26,    29,   488,   697,    13,
              3,    84,    27,    29,    45,    24,     0,     0,     0,     0],
         [ 1623,    63,    14,     6,   100,   869,     7,    97,    13,   856,
             84,    31,    92,     2,   236,     7,    15,    20,     3,    11,
            283,    46,     8,  7216,    82,  3013,     7,   184,     2,   128],
         [  209,   424,    32,  3395,    54,     3,    37,   720,   833,     7,
             15,    20,     9,    31,  8191,     4,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [  149,    29,    32,     2,   302,     7, 29798,     3,    29,    67,
              1,    18,    39,   317,    38,   600,  1955,    14, 16966,   325,
              1,     8,     1, 29798,    18,    25,    26,    74,   783,   378],
         [  121,    24,   660,     7

###   &emsp;&emsp;  = = = = = Attention Over Sentences in the Document = = = = =

In [ ]:
sent_coeffs = model(x_pred)[2]  # get sentence attention coeffs by passing the review to the model - (you need to convert the inout torch tensor)
sent_coeffs = sent_coeffs[0, :, :]

for elt in zip(sent_coeffs[:,0].tolist(),[' '.join(elt) for elt in my_review_text]):
    print(round(elt[0]*100,2),elt[1])

7.01 There 's a sign on The Lost Highway that says : <OOV> SPOILERS <OOV> ( but you already knew that , did n't you ? ) <PAD> <PAD> <PAD> <PAD>
6.41 Since there 's a great deal of people that apparently did not get the point of this movie , I 'd like to contribute my interpretation of why the plot
7.21 As others have pointed out , one single viewing of this movie is not sufficient . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
9.2 If you have the DVD of MD , you can <OOV> ' by looking at David Lynch 's 'Top 10 <OOV> to <OOV> MD ' ( but only upon second
27.04 ; ) First of all , Mulholland Drive is downright brilliant . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
24.69 A masterpiece . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
18.43 This is the kind of movie that refuse

### &emsp;&emsp; = = = = = Attention Over Words in Each Sentence = = = = =

In [ ]:
word_coeffs = model(x_pred)[1]

word_coeffs_list = word_coeffs.reshape(7,30).tolist()

# match text and coefficients:
text_word_coeffs = [list(zip(words,word_coeffs_list[idx][:len(words)])) for idx,words in enumerate(my_review_text)]

for sent in text_word_coeffs:
    [print(elt) for elt in sent]
    print('= = = =')

# sort words by importance within each sentence:
text_word_coeffs_sorted = [sorted(elt,key=operator.itemgetter(1),reverse=True) for elt in text_word_coeffs]

for sent in text_word_coeffs_sorted:
    [print(elt) for elt in sent]
    print('= = = =')

('There', 0.07890594152120202)
("'s", 0.07122610333716808)
('a', 0.07245530573441111)
('sign', 0.05425781945833271)
('on', 0.02513286662931138)
('The', 0.019865352484685798)
('Lost', 0.038951908300717285)
('Highway', 0.08942684185866584)
('that', 0.025482337665762172)
('says', 0.03187431153094438)
(':', 0.01813153980289328)
('<OOV>', 0.014213162164573594)
('SPOILERS', 0.01776470143482485)
('<OOV>', 0.014625565635193176)
('(', 0.01794839445553427)
('but', 0.01655684453186554)
('you', 0.03563011389710826)
('already', 0.02541246840069196)
('knew', 0.04061701416285266)
('that', 0.02407608225677417)
(',', 0.02558711523444284)
('did', 0.022668804993978546)
("n't", 0.03134171425964674)
('you', 0.038068873075081576)
('?', 0.029287644524262094)
(')', 0.019652003079798253)
('<PAD>', 0.023619442182090312)
('<PAD>', 0.024796271442869715)
('<PAD>', 0.025461762155509146)
('<PAD>', 0.0269616936830318)
= = = =
('Since', 0.06362635934546007)
('there', 0.028509046362875726)
("'s", 0.04652008208776166)
(